In [25]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

In [2]:
url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=600'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [3]:
table = soup.find('table')
table

<table class="a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated"><tr><th class="a-text-right mojo-field-type-rank a-nowrap"><span title="Rank">Rank</span>
</th><th class="a-text-left mojo-field-type-title a-nowrap"><span title="Title">Title</span>
</th><th class="a-text-right mojo-field-type-money a-nowrap"><span title="Lifetime Gross">Lifetime Gross</span>
</th><th class="a-text-left mojo-field-type-year a-nowrap"><span title="Year">Year</span>
</th></tr><tr><td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">601</td><td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt3861390/?ref_=bo_cso_table_1">Dumbo</a></td><td class="a-text-right mojo-field-type-money">$114,766,307</td><td class="a-text-left mojo-field-type-year"><a class="a-link-normal" href="/year/2019/?ref_=bo_cso_table_1">2019</a></td></tr><tr><td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">602</

In [4]:
rows = [row for row in table.find_all('tr')]

In [5]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Dumbo': ['/title/tt3861390/?ref_=bo_cso_table_1',
  '601',
  'Dumbo',
  '$114,766,307',
  '2019'],
 'Fifty Shades Darker': ['/title/tt4465564/?ref_=bo_cso_table_2',
  '602',
  'Fifty Shades Darker',
  '$114,581,250',
  '2017'],
 'Dodgeball': ['/title/tt0364725/?ref_=bo_cso_table_3',
  '603',
  'Dodgeball',
  '$114,326,736',
  '2004'],
 'The Village': ['/title/tt0368447/?ref_=bo_cso_table_4',
  '604',
  'The Village',
  '$114,197,520',
  '2004'],
 'Bolt': ['/title/tt0397892/?ref_=bo_cso_table_5',
  '605',
  'Bolt',
  '$114,053,579',
  '2008'],
 'Maleficent: Mistress of Evil': ['/title/tt4777008/?ref_=bo_cso_table_6',
  '606',
  'Maleficent: Mistress of Evil',
  '$113,929,605',
  '2019'],
 'Wild Wild West': ['/title/tt0120891/?ref_=bo_cso_table_7',
  '607',
  'Wild Wild West',
  '$113,804,681',
  '1999'],
 'Night at the Museum: Secret of the Tomb': ['/title/tt2692250/?ref_=bo_cso_table_8',
  '608',
  'Night at the Museum: Secret of the Tomb',
  '$113,746,621',
  '2014'],
 'Magic Mike':

In [6]:
top_movies = pd.DataFrame(movies).T  #transpose
top_movies.columns = ['link_stub', 'rank_all_movies','title', 
                    'lifetime_gross', 'year']

top_movies.shape

(199, 5)

In [7]:
def get_movie_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
        # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [8]:
import dateutil.parser

def money_to_int(moneystring):
    if moneystring:
        moneystring = moneystring.replace('$', '').replace(',', '')
        return int(moneystring)
    else:
        pass

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

def clean_genres(genre_list):
    genres = [item.strip() for item in genre_list if item.strip()]
    return genres 

In [9]:
def clean_sales(lst):
    
    '''Clean the summary domestic/international summary table
    
    Takes the raw list of money strings and converts each item to int
    '''
    
    sale_list = [item.text for item in lst]
    
    sale_list = [money_to_int(item) for item in sale_list]
   
    return sale_list


def clean_regions(lst):
    
    '''Clean the summary domestic/international summary table
    
    Takes raw list of region string, cleans, and puts back into list
    '''
    
    region_list = [item.text for item in lst]
    region_list = [item.strip() for item in region_list if item.split()]
    region_list = [item.split('(')[0].strip() for item in region_list]
    
    return region_list

In [20]:
def get_movie_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic distributor
        - budget
        - domestic gross
        - earliest release
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")

    
    headers = ['movie_title', 'domestic_distributor','budget', 'domestic_opening_sales',
               'earliest_release',
               'rating', 'runtime_minutes','genres','summary_sales','crew_list','cast_list']
    
    #Get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()
    
    #Get domestic distributor
    domdist_regex = re.compile('Domestic Distributor')
    domdist_string = soup.find(text=domdist_regex).findNext().text
    domestic_dist = domdist_string.split('See')[0].strip()
    
    # Domestic distributor
    domestic_dist = get_movie_value(soup,'Domestic Distributor').split('See')[0].strip()

    # Budget
    raw_budget = get_movie_value(soup,'Budget')
    budget = money_to_int(raw_budget)

    # Domestic opening sales
    raw_domestic_opening = get_movie_value(soup,'Domestic Opening')
    domestic_opening_sales = money_to_int(raw_domestic_opening)

    #Earliest release date (raw)
    earliest_release_str = get_movie_value(soup,'Earliest Release').split('\n')[0].strip()
    earliest_release = datetime.strptime(earliest_release_str, '%B %d, %Y')
    earliest_release.strftime('%Y-%m-%d')

    #Rating
    rating = get_movie_value(soup,'MPAA')

    # Runtime
    raw_runtime = get_movie_value(soup,'Running Time')
    runtime_minutes = runtime_to_minutes(raw_runtime)

    #Genres
    raw_genres = get_movie_value(soup, 'Genres').split('\n')
    genres = clean_genres(raw_genres)
    
    #Sales list (domestic, international, worldwide)
    raw_sales_list = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0:]
    sales_list = clean_sales(raw_sales_list)
    
    # Regions list
    raw_sales_regions = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='a-size-small')[0:]
    regions_list = clean_regions(raw_sales_regions)
    
    # Zip region and associated sales per region - believe should solve for any missing regions
    summary_sales = [list(i) for i in zip(regions_list,sales_list)]
    
    #Find link for cast and crew
    credits = soup.find('a', class_='a-size-base a-link-normal mojo-navigation-tab',href=True)
    credits_href = str(credits.get('href'))
    credits_url = base_url + credits_href
    
    #Refer to cast and crew page
    response1 = requests.get(credits_url,headers = user_agent)
    page1 = response1.text
    soup1 = BeautifulSoup(page1, 'lxml')
    
    #Find crew table, cast table
    table_crew = soup1.find('table',id='principalCrew').find_all('td')
    table_cast = soup1.find('table',id='principalCast').find_all('td')
    
    #clean the tables
    def clean_credits(table1):
        lst1 = [item.text for item in table1]
        lst2 = [item.split('\n')[0].strip() for item in lst1]
        lst2 = [item for item in lst2[::2]]

        return lst2

    crew_list = clean_credits(table_crew)
    cast_list = clean_credits(table_cast)
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [title, domestic_dist, budget,domestic_opening_sales,
                                    earliest_release,
                                    rating, runtime_minutes,genres,summary_sales,crew_list,cast_list]))

    return movie_dict

In [21]:
top_movies_page_info_list = []

for link in top_movies.link_stub:
    top_movies_page_info_list.append(get_movie_dict(link))

In [22]:
top_movies_page_info = pd.DataFrame(top_movies_page_info_list)  #convert list of dict to df
top_movies_page_info.set_index('movie_title', inplace=True)

In [23]:
top_movies_page_info.to_csv('mojo_pg4.csv')

In [24]:
top_movies_page_info.head()

,domestic_distributor,budget,domestic_opening_sales,earliest_release,rating,runtime_minutes,genres,summary_sales,crew_list,cast_list
movie_title,,,,,,,,,,
Dumbo,Walt Disney Studios Motion Pictures,170000000.0,45990748.0,"March 27, 2019",PG,112.0,"[Adventure, Family, Fantasy]","[[Domestic, 114766307], [International, 238518...","[Tim Burton, Ehren Kruger, Helen Aberson, Haro...","[Colin Farrell, Michael Keaton, Danny DeVito, ..."
Fifty Shades Darker,Universal Pictures,55000000.0,46607250.0,"February 8, 2017",R,118.0,"[Drama, Romance]","[[Domestic, 114581250], [International, 266964...","[James Foley, Niall Leonard, E.L. James, Dana ...","[Dakota Johnson, Jamie Dornan, Eric Johnson, E..."
Dodgeball,Twentieth Century Fox,20000000.0,30070196.0,"June 18, 2004",PG-13,92.0,"[Comedy, Sport]","[[Domestic, 114326736], [International, 540964...","[Rawson Marshall Thurber, Rawson Marshall Thur...","[Ben Stiller, Christine Taylor, Vince Vaughn, ..."
The Village,Walt Disney Studios Motion Pictures,60000000.0,50746142.0,"August 13, 2003",PG-13,108.0,"[Drama, Mystery, Thriller]","[[Domestic, 114197520], [International, 142500...","[M. Night Shyamalan, M. Night Shyamalan, Sam M...","[Sigourney Weaver, William Hurt, Joaquin Phoen..."
Bolt,Walt Disney Studios Motion Pictures,150000000.0,26223128.0,"November 21, 2008",PG,96.0,"[Adventure, Animation, Comedy, Family, Fantasy]","[[Domestic, 114053579], [International, 195926...","[Byron Howard, Chris Williams, Dan Fogelman, C...","[John Travolta, Miley Cyrus, Susie Essman, Mar..."
